In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp modeBG

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import shap
import pandas as pd
import numpy as np
from boruta import BorutaPy

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

from sksurv.util import Surv
from sksurv.datasets import load_flchain
from sklearn.preprocessing import QuantileTransformer
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import concordance_index_censored
from sksurv.svm import FastSurvivalSVM

from PredictiveMaintenance2 import Datasets,Visualize,FeatureEng,Model

F:\Lalitha\anaconda3\envs\Predictive-Maintenance\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# load pre processed dataset
machineData = pd.read_csv(r'C:\Users\Lalitha\Desktop\GyanData\predictive-maintenance\PredictiveMaintenance2\nbs\Machine_Data_Preprocessed.csv')
machineData.head(2)

,date,device,failure,metric1,metric2,metric3,metric4,metric5,metric6,metric7,metric8,metric9,last_failure,RUL,SurvivalTime
0,2015-01-01,0,0,141503600,0,0,1,19,494462,16,16,3,1,18,1
1,2015-01-01,1,0,55587136,0,0,0,7,199132,0,0,0,1,214,1


In [ ]:
# sort values by device and date
machineData = machineData.sort_values(['device','date'],ascending= True).reset_index(drop=True)
machineData.head(2)

,date,device,failure,metric1,metric2,metric3,metric4,metric5,metric6,metric7,metric8,metric9,last_failure,RUL,SurvivalTime
0,2015-01-01,0,0,141503600,0,0,1,19,494462,16,16,3,1,18,1
1,2015-01-02,0,0,161679800,0,0,1,19,495730,16,16,3,1,17,2


In [ ]:
x = machineData.drop(['date','failure','RUL','SurvivalTime'],axis=1)
y = machineData['RUL']

# split the data
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

# standardize the features
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.fit_transform(x_test)

In [ ]:
# convert train and test dataset to numpy array for applying boruta
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()

In [ ]:
# Instantiate the random forest regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Instantiate the Boruta object
boruta = BorutaPy(estimator=rf, n_estimators='auto', verbose=2, alpha=0.05, max_iter=100, random_state=42)

# Fit the Boruta object to the training data
boruta.fit(x_train, y_train)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	11
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	11
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	11
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	11
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	11
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	11
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	11
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	8
Tentative: 	3
Rejected: 	0
Iteration: 	9 / 100
Confirmed: 	8
Tentative: 	3
Rejected: 	0
Iteration: 	10 / 100
Confirmed: 	8
Tentative: 	3
Rejected: 	0
Iteration: 	11 / 100
Confirmed: 	8
Tentative: 	3
Rejected: 	0
Iteration: 	12 / 100
Confirmed: 	8
Tentative: 	3
Rejected: 	0
Iteration: 	13 / 100
Confirmed: 	8
Tentative: 	3
Rejected: 	0
Iteration: 	14 / 100
Confirmed: 	8
Tentative: 	3
Rejected: 	0
Iteration: 	15 / 100
Confirmed: 	8
Tentative: 	3
Rejected: 	0
Iteration: 	16 / 100
Confirmed: 	8
Tentative: 	3
Rejected: 	0
Iteration:

BorutaPy(estimator=RandomForestRegressor(n_estimators=46,
                                         random_state=RandomState(MT19937) at 0x2A410022240),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x2A410022240, verbose=2)

In [ ]:
# Get the selected features
selected_features = boruta.support_

# Train a regression model using the selected features
x_train_sel = x_train[:, selected_features]
x_test_sel = x_test[:, selected_features]

In [ ]:
# Create a Random Forest Regressor model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model to the training data
rf_model.fit(x_train_sel, y_train)

RandomForestRegressor(random_state=42)

In [ ]:
# Make predictions on the testing data
y_pred = rf_model.predict(x_test_sel)

In [ ]:
# Calculate the Mean Squared Error (MSE) and R-squared value of the predictions
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [ ]:
# Print the results
print(f"Mean Squared Error: {mse}")
print(f"R-squared value: {r2}")

Mean Squared Error: 75.41464153709978
R-squared value: 0.9887159768571434


In [ ]:
# Perform cross-validation on the model
scores = cross_val_score(rf_model,x,y, cv=5)
print('Cross-Validation Scores:', scores)
print('Mean Cross-Validation Score:', np.mean(scores))

Cross-Validation Scores: [0.70059999 0.17081497 0.62553341 0.23529984 0.58875504]
Mean Cross-Validation Score: 0.46420064767633457


# Probability of failure in next N days

In [ ]:
machineData = pd.read_csv(r'C:\Users\Lalitha\Desktop\GyanData\predictive-maintenance\PredictiveMaintenance2\nbs\Machine_Data_Preprocessed.csv')

# sort values by device and date
machineData = machineData.sort_values(['device','date'],ascending= True).reset_index(drop=True)

# get last record of each device
machineData['date'] = pd.to_datetime(machineData['date'])
last_date = machineData.groupby('device')['date'].transform(max) == machineData['date']

# Filter the data to keep only the rows with the maximum observation date for each device
machineData = machineData[last_date].reset_index(drop=True)

# Print the final output
print(machineData.head(2))

        date  device  failure    metric1  metric2  metric3  metric4  metric5   
0 2015-01-19       0        1   64499464        0        0        1       19  \
1 2015-08-03       1        1  110199904      240        0        0        8   

   metric6  metric7  metric8  metric9  last_failure  RUL  SurvivalTime  
0   514661       16       16        3             1    0            19  
1   294852        0        0        0             1    0           215  


In [ ]:
"""
# convert data into survival format
x = machineData.drop(['date', 'failure', 'RUL','SurvivalTime'], axis=1)
y = Surv.from_arrays(machineData['failure'], machineData['SurvivalTime'])
"""

"\n# convert data into survival format\nx = machineData.drop(['date', 'failure', 'RUL','SurvivalTime'], axis=1)\ny = Surv.from_arrays(machineData['failure'], machineData['SurvivalTime'])\n"

In [ ]:
# or
# Create a structured array for the survival analysis
x = machineData.drop(['date', 'failure', 'RUL','SurvivalTime'], axis=1)
y = np.zeros(len(machineData), dtype=[('failure', 'bool'), ('SurvivalTime', 'float')])

y['failure'] = machineData['failure'].astype(bool)
y['SurvivalTime'] = machineData['SurvivalTime'].astype(float)

In [ ]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
# Normalize the data using quantile normalization
# n_quantiles = min(1000,machineData.shape[0])
qn = QuantileTransformer(output_distribution='normal', random_state=42)
x_train_norm = qn.fit_transform(x_train)
x_test_norm = qn.transform(x_test)

n_quantiles (1000) is greater than the total number of samples (109). n_quantiles is set to n_samples.


In [ ]:
# feature selection

In [ ]:
#| import
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest

In [ ]:
# Train the Cox PH survival model
estimator = CoxPHSurvivalAnalysis(alpha=0.05)
estimator.fit(x_train_norm, y_train)

CoxPHSurvivalAnalysis(alpha=0.05)

In [ ]:
# Perform feature selection
selector = SelectKBest(k=6)
selector.fit(x_train_norm, y_train)

SelectKBest(k=6)

In [ ]:
# Get the selected features
selected_features = x_train.columns[selector.get_support()]
selected_features

Index(['metric3', 'metric4', 'metric5', 'metric6', 'metric7', 'metric8'], dtype='object')

In [ ]:
# Train the Cox PH survival model
estimator = CoxPHSurvivalAnalysis(alpha=0.05)
estimator.fit(x_train_norm, y_train)

CoxPHSurvivalAnalysis(alpha=0.05)

In [ ]:
# I need a older version of numpy to work with < 1.20 
"""
# initialize the SHAP explainer for the Cox model
explainer = shap.Explainer(estimator, x_train)

# generate SHAP values for the testing data
shap_values = explainer(x_test)

# summarize the feature importance across all samples
shap.summary_plot(shap_values, x_test)
"""

'\n# initialize the SHAP explainer for the Cox model\nexplainer = shap.Explainer(estimator, x_train)\n\n# generate SHAP values for the testing data\nshap_values = explainer(x_test)\n\n# summarize the feature importance across all samples\nshap.summary_plot(shap_values, x_test)\n'

In [ ]:
# Predict the probability of failure in the next n days for each machine in the testing set
y_pred = estimator.predict(x_test_norm)

# Evaluate the performance of the model
c_index = concordance_index_censored(y_test['failure'], y_test['SurvivalTime'], y_pred)
print(f"C-index: {c_index}")

C-index: (0.7244582043343654, 234, 89, 0, 0)


c- index is 0.7 indicate good performance of the survival model

In [ ]:
# Predict the survival functions for the testing set
survival_functions = estimator.predict_survival_function(x_test_norm)

In [ ]:
print(len(survival_functions))

28


In [ ]:
#print(survival_functions[0])

In [ ]:
# Get the probability of failure in the next n days
n_days = 100 
failure_prob = []

for sf in survival_functions:
    t_idx = np.argmin(np.abs(sf.x - n_days))
    failure_prob.append(1 - sf.y[t_idx])
    
failure_prob = np.array(failure_prob)

# Print the probability of failure for each device in the testing set
for i, prob in enumerate(failure_prob):
    if prob>0.5:
        print(f"----Device-{i} : {prob*100:.2f}-----")
    else:
        print(f"Device-{i} : {prob*100:.2f}")  

----Device-0 : 60.24-----
----Device-1 : 58.94-----
Device-2 : 0.00
Device-3 : 30.58
Device-4 : 0.00
Device-5 : 0.00
Device-6 : 0.01
Device-7 : 44.11
Device-8 : 36.54
----Device-9 : 76.97-----
----Device-10 : 87.75-----
Device-11 : 40.30
Device-12 : 26.68
Device-13 : 0.00
Device-14 : 31.15
----Device-15 : 51.14-----
----Device-16 : 62.40-----
----Device-17 : 93.28-----
Device-18 : 25.45
Device-19 : 0.00
Device-20 : 0.00
Device-21 : 37.59
Device-22 : 0.00
Device-23 : 40.37
Device-24 : 28.94
Device-25 : 41.13
Device-26 : 32.96
Device-27 : 39.22


Calculate the probability of failure in the next n days using the predicted hazard ratios and the survival function

# using kaplan-Meier survival function from lifelines

In [ ]:
#| import
from lifelines import KaplanMeierFitter
from lifelines.utils import median_survival_times

In [ ]:
"""
# Prepare the data for Kaplan-Meier model

train_frac = 0.7
train_size = int(train_frac * len(machineData))
train_data = machineData[:train_size]
test_data = machineData[train_size:]


kmf = KaplanMeierFitter()
kmf.fit(train_data['SurvivalTime'],train_data['failure'])

n_days = 100

# Predict the survival function using Kaplan-Meier model
sf = kmf.survival_function_at_times(np.array([n_days / 365.25]))

# Get the probability of failure in next n days for each device
prob_failure = []
for i, row in test_data.iterrows():
    if row['SurvivalTime'] <= n_days:
        prob_failure.append(1.0)
    else:
        median_time = median_survival_times(kmf.predict(row['SurvivalTime']))
        prob_failure.append(1 - sf[median_time])

# Compute the concordance index to evaluate the model performance
c_index = concordance_index_censored(test_data['failure'], test_data['SurvivalTime'], prob_failure)
print("Concordance index:", c_index)

time_points = [1, 2, 3]  # n = 1 year, 2 years, 3 years
sf = kaplan_meier_estimator(y_test["failure"], y_pred, y_test["SurvivalTime"])

prob_fail = 1 - sf[0](time_points)
"""

'\n# Prepare the data for Kaplan-Meier model\n\ntrain_frac = 0.7\ntrain_size = int(train_frac * len(machineData))\ntrain_data = machineData[:train_size]\ntest_data = machineData[train_size:]\n\n\nkmf = KaplanMeierFitter()\nkmf.fit(train_data[\'SurvivalTime\'],train_data[\'failure\'])\n\nn_days = 100\n\n# Predict the survival function using Kaplan-Meier model\nsf = kmf.survival_function_at_times(np.array([n_days / 365.25]))\n\n# Get the probability of failure in next n days for each device\nprob_failure = []\nfor i, row in test_data.iterrows():\n    if row[\'SurvivalTime\'] <= n_days:\n        prob_failure.append(1.0)\n    else:\n        median_time = median_survival_times(kmf.predict(row[\'SurvivalTime\']))\n        prob_failure.append(1 - sf[median_time])\n\n# Compute the concordance index to evaluate the model performance\nc_index = concordance_index_censored(test_data[\'failure\'], test_data[\'SurvivalTime\'], prob_failure)\nprint("Concordance index:", c_index)\n\ntime_points = [1, 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()